# Working on H1: the 3000 samples from Farm 1

#### A note on converting Jupyter Notebook output to MS Word Documents

- The best way to convert the ipynb file (Jupyter Notebook) to a docx file is to follow the two step approach explained in: https://blog.ouseful.info/2017/06/13/using-jupyter-notebooks-for-assessment-export-as-word-docx-extension/, i.e. run the following two commands on the anaconda command line
- Step 1: $ jupyter nbconvert --no-input --to html file_name.ipynb (use --no-input to exclude code cells, i.e. convert only markdown cells)

- Step 2: $ pandoc -s file_name.html -o file_name.docx
- This approach produces, by far, the best quality docx ouput, no distortion of either the text or the graphs.  The only drawback is that it include the hidden code cells made by the "hide all" extension.  I have to manually delete those contents from the produced docx file.
#### convert ijynb to a docx file
- https://nbconvert.readthedocs.io/en/latest/index.html
- install nbconvert [pip install nbconvert]
- install pandoc: [https://github.com/jgm/pandoc/releases/tag/3.1.3]

In [1]:
import sys

oneDrive_root={}
oneDrive_root[1]="C:\\Users\\Chihyang\OneDrive for Business\\"
oneDrive_root[2]="C:\\Users\\Chihyang\OneDrive for Business\\"
oneDrive_root[3]="C:\\Users\\tsaic\\OneDrive - State University of New York at New Paltz\\"  # laptop

site=3   # the short or long business OneDrive directory name
lib_dir=oneDrive_root[site]+'\\Prudentia\\Tsaipy'
# append additional library path for this study
sys.path.append(lib_dir)

In [2]:
import pandas as pd
import numpy as np
#import sklearn as sk
#from sklearn.cluster import KMeans
from dbconnect.mongodb import cursor_to_dataframe3
from WFN_lib.WindFarm import distance2center,  write_df_sub2Excel
from WFN_lib.mongodb_util import flatten_dictionary
from WFN_lib.cluster_classify import cluster_analysis


In [3]:
from pymongo import MongoClient
import pymongo
from dbconnect import mongodb as mdb

<div class="alert alert-block alert-info"><b>MongoDb Parameters:</b>

</div>

In [4]:
## pymongo connect to mongodb database and collection
db_name = "Windfarm_6000"
client = MongoClient('localhost', 27017)  # connect to the db engine
db = client[db_name]
coll_data=db['Data_S6000']
coll_res = db['Results_S6000']
print(f"Rows in coll_data = {coll_data.count_documents({})}")
print(f"Rows in coll_res = {coll_res.count_documents({})}")

Rows in coll_data = 6000
Rows in coll_res = 6000


<div class="alert alert-block alert-warning"><b>I. Clustering the 3000 cases in H1</b>

<b>DO NOT</b> use <b>v31_minmax</b> obtained from S6000 since it is scaled with the whole 6000 cases.  <b>For validation purpose</b> with H2 files, we need to scale v31 with only the 3000 cases in H1.

Thus, for the S3000 database, we only store v31_minmax variables scaled based on the 3000 files in H1.  If you need variable values in the original scale, just retrieve them from collection <b>Data_S6000</b> with the indices H1_xxxx, where xxxx ranges from <b>H1_0001</b> to <b>H1_3000</b>.
</div>

### Extract data from H1

In [5]:
res = coll_data.find({'site':'H1'},{'v31':1})  # index '_id' is automatically added
df_H1 = cursor_to_dataframe3(res,"_id")
print(df_H1.shape)
print(df_H1.head(3))

(3000, 31)
         v31.spectralCentroid  v31.spectralCrest  v31.spectralDecrease  \
H1_0001             63.081251          28.079092            -92.466205   
H1_0002             57.650009          30.680069           -259.422066   
H1_0003             75.049887          25.424498            -22.843544   

         v31.spectralEntropy  v31.spectralFlatness  v31.spectralFlux  \
H1_0001             0.143241              0.010610      1.087143e-07   
H1_0002             0.067829              0.011763      4.547078e-05   
H1_0003             0.217764              0.008283      3.236673e-06   

         v31.spectralKurtosis  v31.spectralRolloffPoint  v31.spectralSkewness  \
H1_0001           1420.161272                115.968033             25.345770   
H1_0002            847.458521                 72.747519             22.280157   
H1_0003            180.938195                177.970026              9.196154   

         v31.spectralSlope  ...      v31.PR  v31.Fo  v31.AMfactor   v31.DAM  \

In [6]:
## use the unscaled data and ask the function to scale it based on the 3000 files
## df_res is the dataframe with the cluster allotment; df_scaled is the scaled data
## Save the trained KMeans model to the directory save_model_dir
df_res, df_H1_scaled=cluster_analysis(df_H1, [3,4,5,6], random_state=42, scaler_type="MinMax",n_init=10, max_iter=300, tol=0.0001,
                                   save_model_dir='.//trained_models//', model_file_name='Kmeans_H1_S3000_v31_MinMax_K=', 
                                   scaler_model_name='_scaler_v31_H1_S3000')

In [7]:
print(df_res.head(3))
print(df_H1_scaled.head(3))

         K=3  K=4  K=5  K=6
H1_0001    1    2    0    3
H1_0002    2    1    1    1
H1_0003    0    2    0    3
         v31.spectralCentroid  v31.spectralCrest  v31.spectralDecrease  \
H1_0001              0.017638           0.815540              0.996951   
H1_0002              0.011442           0.939524              0.991424   
H1_0003              0.031291           0.689000              0.999256   

         v31.spectralEntropy  v31.spectralFlatness  v31.spectralFlux  \
H1_0001             0.228597              0.029063          0.001635   
H1_0002             0.105126              0.032293          0.684277   
H1_0003             0.350613              0.022545          0.048706   

         v31.spectralKurtosis  v31.spectralRolloffPoint  v31.spectralSkewness  \
H1_0001              0.006851                  0.023762              0.077191   
H1_0002              0.004065                  0.008786              0.067042   
H1_0003              0.000823                  0.045246    

In [8]:
df_H1_clusters, cluster_size_H1, cluster_means_H1, cluster_vars_H1 = distance2center(df_H1_scaled, df_res, [3,4,5,6])

In [9]:
## df_H1: for each sample, this dataframe contains the cluster number, the distance to the cluster center, 
##   and the ranking of distance to cluster center (closest to farthest).  
## !! Note that measure of the distance and ranking are made with respect to the cluster the sample is in.
print(df_H1_clusters.shape)
df_H1_clusters.head(3)

(3000, 12)


,K=3,K=4,K=5,K=6,dist_K=3,rank_K=3,dist_K=4,rank_K=4,dist_K=5,rank_K=5,dist_K=6,rank_K=6
H1_0001,1,2,0,3,0.539937,1383,0.552750,934,0.565479,980,0.557639,973
H1_0002,2,1,1,1,0.557416,807,0.585724,740,0.589200,730,0.409207,328
H1_0003,0,2,0,3,0.603725,109,0.601751,966,0.613486,1006,0.609877,997


<div class="alert alert-block alert-info"><b>Write the K=5 Clustering result with additional details into an Excel file:</b> Data in a cluster is presented in one Worksheet of the file.   

In each cluster, rows are sorted from the nearest to its class center the farthest.  For each row (case), columns include 
<ol>
<li> Cluster it is assigned to,</li>
<li>distance to cluster center,</li>
<li>ranking of distance to center,</li>
<li>two IOA ratings (prominenace and modulation depth) for each band (four bands), and</li> 
<li>YAMnet top five classes with corresponding probabilities (10 columns)</li>
</ol>

<b>Cluster Characteristics</b>:
<ul>
<li>Cluster 0: 1061 cases; Low confidence even to the first YAMnet class,</li>
<li>Cluster 1: 837 cases; Low confidence even to the first YAMnet class,</li>
<li>Cluster 2: 58 cases; High confidence to the first YAMnet class; first class has lots of "silence"</li>
<li>Cluster 3: 814 cases; Moderate confidence to the first YAMnet class; first class has lots of "silence", and</li> 
<li>Cluster 4: 230 cases; Moderate confidence to the first YAMnet class; first class has lots of "Animal"</li>
</ul>
</div>

In [10]:
## First, filter the columns to keep only those K=5 columns
k=5
## get the columns whose names contain K=5
df_H1_K = df_H1_clusters.filter(regex='K='+str(k))
#print(df_H1_K.head(2))

## Retrieve IOA rating and YAMnet classification data from S6000
res=db['Results_S6000'].find({'_id':{'$in':list(df_H1_clusters.index)}},{'IOA_rating':1, 'YAMnet':1})
#print(db['Results_S6000'].index_information())
#print(res.count())
df_IOA_YAM_S6000 = cursor_to_dataframe3(res,"_id")
print("Details retrieved from S6000 has shape: ",df_IOA_YAM_S6000.shape)

## Join the clustering result dataframe (3000 cases) to the IOA-YAMnet detail dataframe (from S6000) on index
df_H1K_IOA_YAM = df_H1_K.join(df_IOA_YAM_S6000, how='inner')
print("Joint dataframe has shape: ",df_H1K_IOA_YAM.shape)
print(df_H1K_IOA_YAM.head(2))
## write the dataframe to an Excel file,  one worksheet for each cluster 
#write_df_sub2Excel(df_H1K_IOA_YAM, method='', n_clusters=k, filename=f'.\\Results_H1\\S3000_Results\\H1_S3000_K={k}_IOA_YAM.xlsx')


Details retrieved from S6000 has shape:  (3000, 18)
Joint dataframe has shape:  (3000, 21)
         K=5  dist_K=5  rank_K=5  IOA_rating.025_100Hz.prominence  \
H1_0001    0  0.565479       980                        14.307293   
H1_0002    1  0.589200       730                         5.458717   

         IOA_rating.025_100Hz.L5-L95  IOA_rating.050_200Hz.prominence  \
H1_0001                     3.114064                        25.675691   
H1_0002                     3.831529                         4.284061   

         IOA_rating.050_200Hz.L5-L95  IOA_rating.100_400Hz.prominence  \
H1_0001                     1.621372                        36.235453   
H1_0002                     3.762765                         1.733793   

         IOA_rating.100_400Hz.L5-L95  IOA_rating.200_800Hz.prominence  ...  \
H1_0001                     1.477713                        50.222898  ...   
H1_0002                     2.792567                         2.223483  ...   

         YAMnet.class.top_

In [11]:
print(df_H1_K.shape)
print(df_H1_K.head(3))
print(df_H1_K.groupby('K=5').size())

(3000, 3)
         K=5  dist_K=5  rank_K=5
H1_0001    0  0.565479       980
H1_0002    1  0.589200       730
H1_0003    0  0.613486      1006
K=5
0    1061
1     837
2      58
3     814
4     230
dtype: int64


<div class="alert alert-block alert-success"><b>### Create a MongoDB collection to store H1 result</b> need to load info from df_H1</div>

In [12]:
## Drop the collection, coll_res_H1 and recreate it anew.
coll_res_H1 = 'Data_Results_H1_S3000'
try:
    db.validate_collection(coll_res_H1)  # Try to validate a collection   
    db[coll_res_H1].drop()
    print(f"Collection (table), {coll_res_H1}, dropped")
except pymongo.errors.OperationFailure:  # If the collection doesn't exist
        print(f"Collection, {coll_res_H1}, doesn't exist") 
     
coll_res_H1 = db[coll_res_H1]
### Insert a new row to describe the data in collection_structure
coll_struc=db['varStructures']
coll_struc.delete_one({"_id":"V31_H1_minmax"})
coll_struc.insert_one({"_id":"V31_H1_minmax","category":"scaled_data","file_name":"None","directory":"None",
                           "type":"31 minmax feature variables and clustering information","in_collection":"Data_Results_H1_S3000",
                       "desciption":"Minmax scaled the 31 featured variables using the 3000 files from H1.  Clustering result and distance to center are included."})

Collection, Data_Results_H1_S3000, doesn't exist


In [13]:
K_columns = df_H1.columns
V_columns = df_H1_scaled.columns
## Remove the prefix V31. from the column names
nV_columns = [col[col.find(".")+1:] for col in V_columns]   # new column names after removing the prefix V31.

for i in range(len(df_H1)):
    row = dict()
    row['_id']=df_H1.index[i]
    row['v31_minmax']=dict()
    row['cluster_v31_minmax']=dict()
    ### the 31 minmax scaled variables, scaled based on the S3000 samples      
    # since we stripped the prefix V31. from the column names, need to use both in the loop
    for old_col, new_col in zip(V_columns, nV_columns):
        row['v31_minmax'][new_col]=df_H1_scaled.iloc[i][old_col]
    ### the clustering result and distance to center with ranking
    for col in K_columns:
        row['cluster_v31_minmax'][col]=df_H1.iloc[i][col]  
    if i<= 2: print(row)
    coll_res_H1.insert_one(row)    

{'_id': 'H1_0001', 'v31_minmax': {'spectralCentroid': 0.017637594005318696, 'spectralCrest': 0.8155397082188848, 'spectralDecrease': 0.9969509741955754, 'spectralEntropy': 0.22859728551935327, 'spectralFlatness': 0.0290629742847957, 'spectralFlux': 0.0016345910289633525, 'spectralKurtosis': 0.006851094334737899, 'spectralRolloffPoint': 0.023762321379716456, 'spectralSkewness': 0.07719102975275033, 'spectralSlope': 0.9974095650903214, 'spectralSpread': 0.05435743810758327, 'pitch': 1.0, 'harmonicRatio': 0.016149689583707976, 'LA': 0.2167899481574682, 'ratioLGLA': 0.46005860861977876, 'ratioLCLA': 0.38362688786167887, 'diffLCLA': 0.458930768861343, 'peakloc': 0.85, 'peakval': 0.00019967903502990527, 'pos_slope': 0.008018907555842584, 'neg_slope': 0.9887173364902124, 'PR': 0.06481679674502112, 'Fo': 1.0, 'AMfactor': 0.04705981730750444, 'DAM': 0.015886152932799373, 'peakloc_unweightedSPL': 0.0, 'L63': 0.0468392604214644, 'L125': 0.03449379302729967, 'L250': 0.04006677762574156, 'L500': 0.

<div class="alert alert-block alert-info"><b>Classification:</b> A three step approach to determine and train a classification model to separate the clusters.  This is just a test run.  Currently, no analysis is based on this classification.
<ul>
<li>Based on the 3000 samples in H1.</li>
<li>Use the clustering result (K=k) on v31_minmax on the 3000 samples as the y variables.</li>
<li>Train a SVC classifier to classify the clusters.</li>
</ul>
</div>

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import joblib

#### Step 1: Use GridSearch to determine the hyperparameters

In [15]:
## Set df_scaled as X and df_res["K=5'"] as y
X = df_H1_scaled
y = df_res["K=5"]
## Grid search for the best hyperparameters for SVC
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['linear', 'rbf']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(X, y)
print(grid.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}


#### Step 2: Use a ten-fold cross-validation to evaluate the model

In [16]:
## Test the best hyperparameters using cross-validation
n_folds=10
model = SVC(**grid.best_params_)
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(f"Average Accuracy from the {n_folds} trials: {scores.mean():.3f}  with standard deviation: {scores.std():.3f}")

Average Accuracy from the 10 trials: 0.984  with standard deviation: 0.010


#### Step 3: Fit the model with the best hyperparameters and save the trained model

In [17]:
## Use the best hyperparameters to fit SVC and save the trained model
model.fit(X, y)
## Save the trained model
joblib.dump(model, './/trained_models//SVC_H1_S3000_v31_minmax.joblib')
print("Model saved")

y_pred = model.predict(X)
print(f"Accuracy: {accuracy_score(y, y_pred):.3f}")
print(confusion_matrix(y, y_pred))


Model saved
Accuracy: 0.996
[[1056    1    0    3    1]
 [   2  832    0    0    3]
 [   0    0   58    0    0]
 [   0    1    0  813    0]
 [   0    0    0    1  229]]


In [18]:
## Retrive the saved model and use it to classify the data
model = joblib.load('.//trained_models//SVC_H1_S3000_v31_minmax.joblib')
y_pred = model.predict(X)
print(f"Accuracy: {accuracy_score(y, y_pred):.3f}")

Accuracy: 0.996
